<a href="https://colab.research.google.com/github/valeradiego/EasyTestGit/blob/main/Copy_of_AlgoritmoAvanceTesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.linalg import expm
from scipy.optimize import least_squares
import os

def extract_euler_angles(U):
    # Definir el sistema de ecuaciones no lineales como una función objetivo
    def equations(vars):
        gamma, beta,alpha = vars
        eq1 = np.cos(gamma/2)*np.cos(beta/2)*np.cos(alpha/2)-np.sin(gamma/2)*np.cos(beta/2)*np.sin(alpha/2) - U[0,0].real
        eq2 = np.cos(gamma/2)*np.sin(beta/2)*np.sin(alpha/2)-np.sin(gamma/2)*np.sin(beta/2)*np.cos(alpha/2) - U[0,1].imag
        eq3 = np.cos(gamma/2)*np.cos(beta/2)*np.sin(alpha/2)+np.sin(gamma/2)*np.cos(beta/2)*np.cos(alpha/2) + U[0,1].real
        eq4 = np.cos(gamma/2)*np.sin(beta/2)*np.cos(alpha/2)+np.sin(gamma/2)*np.sin(beta/2)*np.sin(alpha/2) + U[0,0].imag
        return [eq1, eq2, eq3, eq4]

    # Definir las restricciones en las variables
    bounds = ([-np.pi, -np.pi, -np.pi], [np.pi, np.pi, np.pi])

    # Proporcionar una estimación inicial para las soluciones
    initial_guess = [0, 0, 0]

    # Resolver el sistema de ecuaciones con restricciones
    result = least_squares(equations, initial_guess, bounds=bounds)

    # Mostrar el resultado
    if result.success:
        solution = result.x
        print("Soluciones:")
        print("gamma =", solution[0])
        print("beta =", solution[1])
        print("alpha =", solution[2])
    else:
        print("No se encontró una solución.")
    return solution[0], solution[1], solution[2]

# Definir la función para calcular la transpuesta conjugada (adjunta)
def daga(matrix):
    return np.conjugate(matrix).T

# Definir la función para el producto tensorial de dos estados
def tensor_product(state1, state2):
    return np.kron(state1, state2)
import numpy as np

# Definir una función para crear la base de un sistema bipartito
def bipartite_base(base1, base2):
    bipartite_states = []
    for state1 in base1:
        for state2 in base2:
            bipartite_states.append(tensor_product(state1, state2))
    return bipartite_states

# Definir la función para la transformación de estados bipartitos
def matrix_bipartite_transform(base_out, base_in):
    S = 0
    for i in range(len(base_out)):
        S += np.dot(base_out[i], daga(base_in[i]))
    return S

def matrix_break(S):
    Srr=S[0:2,0:2]
    Srl=S[0:2,2:4]
    Slr=S[2:4,0:2]
    Sll=S[2:4,2:4]
    return Srr, Srl, Slr, Sll

def euler_to_englert(gamma, beta, alpha):
    gamma_pol=gamma/2-3*np.pi/4
    alpha_pol=-alpha/2+np.pi/4
    beta_pol=beta/4+gamma_pol/2+alpha_pol/2
    return gamma_pol, beta_pol, alpha_pol

def QWP(theta):
    return 1/np.sqrt(2)*np.array([[1 - 1j * np.cos(2*theta), -1j * np.sin(2*theta) ], [-1j * np.sin(2*theta) , 1 + 1j * np.cos(2*theta)]])

def HWP(theta):
    return np.dot(QWP(theta),QWP(theta))

def Upol(gamma,beta,alpha,phase=1):
    return np.dot(QWP(gamma),np.dot(HWP(beta),QWP(alpha)))*phase

def Vtry(gamma,beta,alpha,phase):
    return expm(-1j*gamma/2*sigma_y)@expm(-1j*beta/2*sigma_z)@expm(-1j*alpha/2*sigma_y)*phase

def MZinterferometer(Mz1, Mz2, Mzr, Mzl, x1, x2):
    # Crear el contenido a imprimir y escribir en el archivo
    content = (
        f"Las características de nuestro interferómetro MZ para medir las probabilidades del sistema bipartito P+ {x1} + {x2}, P+ {x1} - {x2}, P- {x1} + {x2}, P- {x1} - {x2} son:\n"
        f". V1= {Mz1[4]},\n gamma1= {Mz1[0]:.4f},\n beta1= {Mz1[1]:.4f},\n alpha1= {Mz1[2]:.4f},\n phase1= {Mz1[3]:.4f}\n"
        f". V2= {Mz2[4]},\n gamma2= {Mz2[0]:.4f},\n beta2= {Mz2[1]:.4f},\n alpha2= {Mz2[2]:.4f},\n phase2= {Mz2[3]:.4f}\n"
        f". Vr= {Mzr[4]},\n gammar= {Mzr[0]:.4f},\n betar= {Mzr[1]:.4f},\n alphar= {Mzr[2]:.4f},\n phaser= {Mzr[3]:.4f}\n"
        f". Vl= {Mzl[4]},\n gammal= {Mzl[0]:.4f},\n betal= {Mzl[1]:.4f},\n alphal= {Mzl[2]:.4f},\n phasel= {Mzl[3]:.4f}"
    )

    # Imprimir el contenido en la consola
    print(content)

    # Obtener la ruta actual de trabajo
    current_directory = os.getcwd()
    print(f"Guardando archivo en: {current_directory}")

    # Definir el nombre del archivo
    filename = f"MZ_{x1}_{x2}.txt"
    filepath = os.path.join(current_directory, filename)

    # Escribir el contenido en un archivo de texto
    with open(filepath, 'w') as file:
        file.write(content)


# Definir los estados |+z> y |-z>
plus_z = np.array([[1], [0]])
minus_z = np.array([[0], [1]])
base_z = [plus_z, minus_z]

# Generar el estado |+x> y |-x> a partir de |+z> y |-z>
plus_x = 1/np.sqrt(2) * (plus_z + minus_z)
minus_x = 1/np.sqrt(2) * (plus_z - minus_z)
base_x = [plus_x, minus_x]

# Generar el estado |+y> y |-y> a partir de |+z> y |-z>
plus_y = 1/np.sqrt(2) * (plus_z + 1j * minus_z)
minus_y = 1/np.sqrt(2) * (plus_z - 1j * minus_z)
base_y = [plus_y, minus_y]

#Definir matrices de Pauli
sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)

In [ ]:
base_zz=bipartite_base(base_z,base_z)

base_zx=bipartite_base(base_z,base_x)
base_zy=bipartite_base(base_z,base_y)
base_xz=bipartite_base(base_x,base_z)
base_xx=bipartite_base(base_x,base_x)
base_xy=bipartite_base(base_x,base_y)
base_yz=bipartite_base(base_y,base_z)
base_yx=bipartite_base(base_y,base_x)
base_yy=bipartite_base(base_y,base_y)



In [ ]:
#Hallamos la transformación correspondiente a base_xx

Sxx=matrix_bipartite_transform(base_zz,base_xx)

#Comprobamos
np.dot(Sxx,base_xx)

array([[[1.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [1.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [1.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [1.]]])

In [ ]:
#Construimos las martices Srr, Srl, Slr y Sll

Srr, Srl, Slr, Sll=matrix_break(Sxx)
Srr,Srl,Slr,Sll

(array([[ 0.5,  0.5],
        [ 0.5, -0.5]]),
 array([[ 0.5,  0.5],
        [ 0.5, -0.5]]),
 array([[ 0.5,  0.5],
        [ 0.5, -0.5]]),
 array([[-0.5, -0.5],
        [-0.5,  0.5]]))

In [ ]:
#Verificamos condiciones de S

Cond1=np.dot(np.conjugate(Srr.T),Srr)+np.dot(np.conjugate(Slr.T),Slr)
Cond2=np.dot(np.conjugate(Sll.T),Sll)+np.dot(np.conjugate(Srl.T),Srl)
Cond3=np.dot(np.conjugate(Srr.T),Srl)+np.dot(np.conjugate(Slr.T),Sll)
[Cond1,Cond2,Cond3]

[array([[1., 0.],
        [0., 1.]]),
 array([[1., 0.],
        [0., 1.]]),
 array([[0., 0.],
        [0., 0.]])]

In [ ]:
#Hallamos psi, psi_barra, chi, chi_barra, theta, curlytheta y posteriormente sus fases

eigenvalues_psi, eigenvectors_psi = np.linalg.eig(np.dot(daga(Srr),Srr))
eigenvalues_psi, eigenvectors_psi

(array([0.5, 0.5]),
 array([[1., 0.],
        [0., 1.]]))

In [ ]:
curlytheta=np.arccos(np.sqrt(eigenvalues_psi[0]))
theta=np.arccos(np.sqrt(eigenvalues_psi[1]))
psi1=np.asarray([eigenvectors_psi[0]]).T
psi2=np.asarray([eigenvectors_psi[1]]).T


In [ ]:
#Hallamos psi, psi_barra, chi, chi_barra, theta, curlytheta
if np.max(np.abs(Srl))<1e-10:

    curlytheta=0
    theta=0
    psi1=plus_z
    psi2=minus_z

    psi_barra1=np.dot(Srr,psi1)
    psi_barra2=np.dot(Srr,psi2)

    chi1=plus_y
    chi2=minus_y

    chi_barra1=np.dot(Sll,chi1)
    chi_barra2=np.dot(Sll,chi2)
else:
    psi_barra1=np.dot(Srr,psi1)/np.sqrt(eigenvalues_psi[0])
    psi_barra2=np.dot(Srr,psi2)/np.sqrt(eigenvalues_psi[1])
    chi_barra1=1j*np.dot(Slr,psi1)/np.sqrt(1-eigenvalues_psi[0])
    chi_barra2=1j*np.dot(Slr,psi2)/np.sqrt(1-eigenvalues_psi[1])
    chi1=np.dot(daga(Sll),chi_barra1)/np.sqrt(eigenvalues_psi[0])
    chi2=np.dot(daga(Sll),chi_barra2)/np.sqrt(eigenvalues_psi[1])

In [ ]:
psi1

array([[1.],
       [0.]])

In [ ]:
psi_barra1,psi_barra2,chi1,chi2,chi_barra1,chi_barra2

(array([[0.70710678],
        [0.70710678]]),
 array([[ 0.70710678],
        [-0.70710678]]),
 array([[0.-1.j],
        [0.+0.j]]),
 array([[0.+0.j],
        [0.-1.j]]),
 array([[0.+0.70710678j],
        [0.+0.70710678j]]),
 array([[ 0.+0.70710678j],
        [-0.-0.70710678j]]))

In [ ]:
#Creamos las matrices V eligiendo entre 4 posibles combinaciones segun englert
i=1 #1=up 2=down
j=1 #1=up 2=down
signo1=(-1)**(i)
signo2=(-1)**(j)
V1=signo1*1j*np.dot(chi1,daga(psi1))+signo2*1j*np.dot(chi2,daga(psi2))
V2=-signo1*1j*np.dot(psi_barra1,daga(chi_barra1))+-signo2*1j*np.dot(psi_barra2,daga(chi_barra2))
Vr=np.exp(signo1*1j*curlytheta)*np.dot(chi_barra1,daga(chi1))+np.exp(signo2*1j*theta)*np.dot(chi_barra2,daga(chi2))
Vl=np.exp(-signo1*1j*curlytheta)*np.dot(chi_barra1,daga(chi1))+np.exp(-signo2*1j*theta)*np.dot(chi_barra2,daga(chi2))

In [ ]:
Srr_confirm=1/2*np.dot(V2,np.dot(Vr+Vl,V1))
Sll_confirm=1/2*(Vr+Vl)
Srl_confirm=-1j/2*np.dot(V2,Vr-Vl)
Slr_confirm=1j/2*np.dot(Vr-Vl,V1)

np.set_printoptions(precision=5)
print(Srr_confirm,"\n", Srl_confirm,"\n", Slr_confirm,"\n", Sll_confirm)


[[ 0.5+0.j  0.5+0.j]
 [ 0.5+0.j -0.5+0.j]] 
 [[ 0.5-0.j  0.5-0.j]
 [ 0.5-0.j -0.5+0.j]] 
 [[ 0.5+0.j  0.5+0.j]
 [ 0.5+0.j -0.5+0.j]] 
 [[-0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]


In [ ]:
np.set_printoptions(precision=5)
print(Sxx)

[[ 0.5  0.5  0.5  0.5]
 [ 0.5 -0.5  0.5 -0.5]
 [ 0.5  0.5 -0.5 -0.5]
 [ 0.5 -0.5 -0.5  0.5]]


In [ ]:
errorS=np.asarray([np.max(np.abs(Srr-Srr_confirm)),np.max(np.abs(Srl-Srl_confirm)),np.max(np.abs(Slr-Slr_confirm)),np.max(np.abs(Sll-Sll_confirm))])
errorS

array([7.21645e-16, 2.77556e-16, 2.77556e-16, 3.88578e-16])

In [ ]:
V1,V2,Vr,Vl

(array([[-1.+0.j,  0.-0.j],
        [ 0.-0.j, -1.+0.j]]),
 array([[1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]]),
 array([[-0.5+0.5j, -0.5+0.5j],
        [-0.5+0.5j,  0.5-0.5j]]),
 array([[-0.5-0.5j, -0.5-0.5j],
        [-0.5-0.5j,  0.5+0.5j]]))

In [ ]:
#elegir el signo correspondiente segun los errores en V
phase1=-np.sqrt(np.linalg.det(V1))
phase2=np.sqrt(np.linalg.det(V2))
phaser=-np.sqrt(np.linalg.det(Vr))
phasel=-np.sqrt(np.linalg.det(Vl))

In [ ]:
gamma1,beta1,alpha1=extract_euler_angles(V1/phase1)
Vtry1 = Vtry(gamma1,beta1,alpha1,phase1)
print(Vtry1)
print(V1)

Soluciones:
gamma = 0.0
beta = 0.0
alpha = 0.0
[[-1.-0.j  0.-0.j]
 [ 0.-0.j -1.-0.j]]
[[-1.+0.j  0.-0.j]
 [ 0.-0.j -1.+0.j]]


In [ ]:
gamma2,beta2,alpha2=extract_euler_angles(V2/phase2)
Vtry2 = Vtry(gamma2,beta2,alpha2,phase2)
print(Vtry2)
print(V2)

Soluciones:
gamma = 0.0
beta = 0.0
alpha = 0.0
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


In [ ]:
gammar,betar,alphar=extract_euler_angles(Vr/phaser)
Vtryr = Vtry(gammar,betar,alphar,phaser)
print(Vtryr)
print(Vr)

Soluciones:
gamma = 0.7853981577997613
beta = 3.1414378629423916
alpha = -0.7853981689946163
[[-0.50005+0.49995j -0.5    +0.5j    ]
 [-0.5    +0.5j      0.49995-0.50005j]]
[[-0.5+0.5j -0.5+0.5j]
 [-0.5+0.5j  0.5-0.5j]]


In [ ]:
gammal,betal,alphal=extract_euler_angles(Vl/phasel)
Vtryl = Vtry(gammal,betal,alphal,phasel)
print(Vtryl)
print(Vl)

Soluciones:
gamma = 0.7853981745610867
beta = -3.1414378629426656
alpha = -0.7853981522338096
[[-0.50005-0.49995j -0.5    -0.5j    ]
 [-0.5    -0.5j      0.49995+0.50005j]]
[[-0.5-0.5j -0.5-0.5j]
 [-0.5-0.5j  0.5+0.5j]]


In [ ]:
gamma_pol1, beta_pol1, alpha_pol1 = euler_to_englert(gamma1, beta1, alpha1)
gamma_pol2, beta_pol2, alpha_pol2 = euler_to_englert(gamma2, beta2, alpha2)
gamma_polr, beta_polr, alpha_polr = euler_to_englert(gammar, betar, alphar)
gamma_poll, beta_poll, alpha_poll = euler_to_englert(gammal, betal, alphal)
Mz1=[gamma_pol1,beta_pol1,alpha_pol1,phase1,V1]
Mz2=[gamma_pol2,beta_pol2,alpha_pol2,phase2,V2]
Mzr=[gamma_polr,beta_polr,alpha_polr,phaser,Vr]
Mzl=[gamma_poll,beta_poll,alpha_poll,phasel,Vl]

In [ ]:
Upol1=Upol(gamma_pol1,beta_pol1,alpha_pol1,phase1)
Upol2=Upol(gamma_pol2,beta_pol2,alpha_pol2,phase2)
Upolr=Upol(gamma_polr,beta_polr,alpha_polr,phaser)
Upoll=Upol(gamma_poll,beta_poll,alpha_poll,phasel)

In [ ]:
errorV=np.asarray([np.max(np.abs(Upol1-V1)),np.max(np.abs(Upol2-V2)),np.max(np.abs(Upolr-Vr)),np.max(np.abs(Upoll-Vl))])
errorV

array([3.33067e-16, 3.33067e-16, 7.73953e-05, 7.73953e-05])

In [ ]:
MZinterferometer(Mz1, Mz2, Mzr, Mzl, "x", "x")

Las características de nuestro interferómetro MZ para medir las probabilidades del sistema bipartito P+ x + x, P+ x - x, P- x + x, P- x - x son:
. V1= [[-1.+0.j  0.-0.j]
 [ 0.-0.j -1.+0.j]],
 gamma1= -2.3562,
 beta1= -0.7854,
 alpha1= 0.7854,
 phase1= -1.0000-0.0000j
. V2= [[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]],
 gamma2= -2.3562,
 beta2= -0.7854,
 alpha2= 0.7854,
 phase2= 1.0000+0.0000j
. Vr= [[-0.5+0.5j -0.5+0.5j]
 [-0.5+0.5j  0.5-0.5j]],
 gammar= -1.9635,
 betar= 0.3927,
 alphar= 1.1781,
 phaser= -0.7071-0.7071j
. Vl= [[-0.5-0.5j -0.5-0.5j]
 [-0.5-0.5j  0.5+0.5j]],
 gammal= -1.9635,
 betal= -1.1781,
 alphal= 1.1781,
 phasel= -0.7071+0.7071j
Guardando archivo en: c:\Users\valer\OneDrive\Escritorio\Tesis\Matrices
